# 🚀 Selenium Automation Playground s Podmínkami

Tento notebook ukazuje, jak ovládat spouštění buněk pomocí podmínek.

In [ ]:
import os
import csv
from datetime import datetime
import uuid

# --- NASTAVENÍ BĚHU ---
preskocit_cookies = False  # Změň na True pro přeskočení buňky s cookies
pouze_simulace = False     # Změň na True, aby se formulář NEODESLAL

# Pomocná funkce pro zápis do logu
def log_result(rid, msg, st):
    log_file = "automation_log.csv"
    now = datetime.now()
    header = ["id_behu", "datum", "cas", "error_message", "status"]
    new_entry = [rid, now.strftime("%d.%m.%Y"), now.strftime("%H:%M:%S"), msg, st]
    file_exists = os.path.isfile(log_file)
    with open(log_file, mode='a', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        if not file_exists: writer.writerow(header)
        writer.writerow(new_entry)
    print(f"\n📝 Log zapsán: Status={st}, ID={rid}")

run_id = str(uuid.uuid4())[:8]
status = "error"
error_message = "Inicializace notebooku selhala"
print(f"🆔 ID aktuálního běhu: {run_id}")

In [ ]:
status = "error"
error_message = "Nepovedlo se mi otevřít Selenium"

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import sys

print("🔧 Spouštím Chrome...")
chrome_options = Options()
chrome_options.add_argument("--start-maximized")

# --- NASTAVENÍ PRO SERVER (Render/Linux) ---
if sys.platform != "win32":
    print("🐧 Detekován Linux (Render), zapínám headless režim...")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
print("✅ Prohlížeč je připraven.")

In [ ]:
status = "error"
error_message = "Nepovedlo se mi navigovat na web"

print("🌐 Naviguji na web...")
driver.get("https://www.david-kunz-automation.com")
time.sleep(2)
print("✅ Stránka načtena.")

In [ ]:
status = "error"
error_message = "Nepovedlo se mi vyhodnotit podmínku pro cookies"

if not preskocit_cookies:
    try: 
        cookie_button = driver.find_element(By.XPATH, "//button[contains(text(), 'SOUHLASÍM')]")
        if cookie_button.is_displayed():
            cookie_button.click()
            print("✅ Cookies potvrzeny.")
    except:
        print("ℹ️ Cookies banner nenalezen.")
else:
    print("⏭️ Buňka přeskočena na základě nastavení 'preskocit_cookies'")

# Pokud jsme buňku přeskočili schválně, nastavíme status na OK pro tento krok
error_message = ""

In [ ]:
status = "error"
error_message = "Nepovedlo se mi odrolovat na sekci Kontakt"

print("🖱️ Scrollování na kontakt...")
contact_section = driver.find_element(By.ID, "contact-section")
driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", contact_section)
time.sleep(2)
print("✅ Sekce Kontakt nalezena.")

In [ ]:
status = "error"
error_message = "Nepovedlo se mi vyplnit formulář"

print("✍️ Vyplňuji data...")
driver.find_element(By.ID, "firstName").clear()
driver.find_element(By.ID, "firstName").send_keys("Robot")
driver.find_element(By.ID, "lastName").clear()
driver.find_element(By.ID, "lastName").send_keys("Tonda")
driver.find_element(By.ID, "phone").clear()
driver.find_element(By.ID, "phone").send_keys("123456789")
driver.find_element(By.ID, "email").clear()
driver.find_element(By.ID, "email").send_keys("tonda.robot@solutions.cz")
driver.find_element(By.ID, "message").clear()
msg = "Simulovaný běh" if pouze_simulace else "Ostrý běh"
driver.find_element(By.ID, "message").send_keys(f"Chování podle podmínky: {msg}")
print("✅ Vyplněno.")

In [ ]:
status = "error"
error_message = "Nepovedlo se mi odeslat formulář (nebo vyhodnotit podmínku)"

if not pouze_simulace:
    print("🚀 ODESÍLÁM FORMULÁŘ (Ostrý režim)...")
    submit_button = driver.find_element(By.CSS_SELECTOR, "#contact-form button[type='submit']")
    submit_button.click()
    print("✅ Odesláno.")
else:
    print("⏭️ ODEŠLÁNÍ PŘESKOČENO (Režim simulace)")

In [ ]:
status = "error"
error_message = "Nepovedlo se mi počkat před ukončením"
time.sleep(2)
print("✅ Počkáno.")

In [ ]:
status = "error"
error_message = "Nepovedlo se mi zavřít prohlížeč"
driver.quit()
status = "ok"
error_message = ""
log_result(run_id, error_message, status)
print("🏁 Úspěšně dokončeno.")

In [ ]:
if status == "error":
    log_result(run_id, error_message, status)
    print(f"❌ Běh {run_id} byl zaznamenán jako CHYBA.")
else:
    print("ℹ️ Log již byl zapsán jako OK.")